# Layout for Dash App

### Imports

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State, ALL, ALLSMALLER, MATCH

import plotly.express as px
import plotly.graph_objects as go

from stock_functions import stock

In [2]:
# List
html.Datalist(
    id= 'stock-list',
    children= ['VNM', 'BID', 'GAS']
)

indicator_list = ['SMA20', 'SMA50']

In [3]:
#----------Dash initialize----------
app = dash.Dash(__name__,
                external_stylesheets=[dbc.themes.CYBORG],
                meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, innitial-scale=1.0'
                           }]
               )

#------------Main layout------------
app.layout = dbc.Container([
    dbc.Row( # Page Header
        dbc.Col(
            html.H2(
                'Stock Predictor 1.0',
                className= 'text-center text-warning mb-4'
            ),
            width= 12,
            style= {
                'bg-color': '#000000',
                'border-style': 'solid',
                'border-width': '2px',
                'border-color': 'orange'
            }
        )
    ),
    
    dbc.Row( # Control Buttons
        dbc.Col(
            children=[
                dbc.Button('Add Chart', id='add-chart-area', n_clicks=0),
            ]
        )
    ),
    
    dbc.Row( # Graphs
        dbc.Col(
            id= 'chart-area',
            children=[]
        )
    )
], fluid= True)

#-----------Layout Call Back-------------
@app.callback(
    Output('chart-area', 'children'),
    [Input('add-chart-area', 'n_clicks')],
    [State('chart-area', 'children')]
)
def create_chart_area(n_clicks, div_children):
    new_child = html.Div(
        #------------------------
        style={
            'width': '47%',
            'display': 'inline-block',
            'outline': 'thin gray solid',
            'padding': '5px',
            'margin': '15px'
        },
        #---------------------------
        children=[
            dcc.Input(
                style= {'display': 'inline-block'},
                id= {
                    'type': 'stock-id-input',
                    'index': n_clicks
                },
                list= 'stock-list',
                debounce= True,
                placeholder='Input Stock ID'
            ),
            
            dcc.RadioItems(
                style= {'display': 'inline-block'},
                id= {
                    'type': 'chart-type',
                    'index': n_clicks
                },
                options= [
                    {'label': 'Line Chart', 'value': 'line'},
                    {'label': 'OHLC Chart', 'value': 'ohlc'}
                ]
            ),
            
            dcc.Dropdown(
                id= {
                    'type': 'indicators-drop-down',
                    'index': n_clicks
                },
                options=[{'label': x, 'value': x} for x in indicator_list],
                multi=True,
                value= []
            ),
            
            dcc.Graph(
                id= {
                    'type': 'stock-graphs',
                    'index': n_clicks
                },
            )
        ]
    )
    
    div_children.append(new_child)
    return div_children

#---------Display Graph Callback----------
@app.callback(
    Output({'type': 'stock-graphs', 'index': MATCH}, 'figure'),
    [Input({'type': 'stock-id-input', 'index': MATCH}, 'value')],
    [State({'type': 'chart-type', 'index': MATCH}, 'value'),
     State({'type': 'indicators-drop-down', 'index': MATCH}, 'value')]
    
)
def chart_display(stock_id, chart_type, indicators):
# Data source (VNDirect)
    s = stock(f'{stock_id}')
    s.get()
    s.add_indicators(indicators)

# Data draw
    figure = {
        'data': [
            go.Candlestick(
                x = s.join_data.Date,
                low = s.join_data.Low,
                high = s.join_data.High,
                close = s.join_data.Close,
                open = s.join_data.Open,
                increasing_line_color = 'green',
                decreasing_line_color = 'red'
            ),
            px.scatter()
        ],
        'layout': go.Layout(
            title= f'{stock_id}',
            xaxis= {
                'title': 'Date',
                'rangeslider': {'visible': False}
            },
            yaxis= {'title': 'Price'}
        )
    }
    
    return figure
#--------------------------------------
if __name__ == '__main__':
    app.run_server(debug= True, use_reloader= False, port= 3000)

Dash is running on http://127.0.0.1:3000/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
